In [8]:
!pip install robin_stocks pandas
!pip install numpy
!pip install robinstocks
!pip install matplotlib icecream

ERROR: Could not find a version that satisfies the requirement robinstocks (from versions: none)
ERROR: No matching distribution found for robinstocks


In [11]:
from robin_stocks import robinhood as r
import pandas as pd
from datetime import datetime
import logging
# import traceback # for debugging
from sys import exit # for debugging
import traceback # for debugging
import time
import ast
from os import path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from tqdm import tqdm
import numpy as np
from icecream import ic
from colorama import Fore, Back, Style
# Login to Robinhood
r.login('username', 'password')
panic_mode = False # set to True to sell all crypto positions immediately
# List of cryptocurrencies to trade
cryptos = ['BTC', 'ETH', 'ADA', 'DOGE', 'MATIC', 'SHIB', 'ETC', 'UNI', 'AAVE', 'LTC', 'LINK', 'COMP', 'USDC', 'SOL', 'AVAX', 'XLM', 'BCH', 'XTZ']
crypto_data = pd.DataFrame()
# log file
logger = logging.getLogger('crypto_trader')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('crypto_trader.log')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
# import datetime now
from datetime import datetime
# import now function
from datetime import datetime
# import time
import time

now = datetime.now()

# Initialize an empty dataframe to store crypto data
crypto_data = pd.DataFrame()

def get_crypto_data(cryptos, crypto_data):

    for crypto in tqdm(cryptos):
        try:
            #print(f'Getting data for {crypto}, please wait...',end='')
            # Get historical data
            historicals = r.get_crypto_historicals(crypto, interval='day', span='week')
            current_price = historicals[-1]['close_price']
            historical_prices = [x['close_price'] for x in historicals]

            # Load account profile
            profile = r.profiles.load_account_profile()
            buying_power = profile['buying_power']

            # Get crypto positions
            positions = r.crypto.get_crypto_positions()
            print(f'found {len(positions)} positions.')
            for position in tqdm(positions):
                # print(position)
                if position['currency']['code'] == crypto:
                    pos_dict = position['currency']
                    min_order_size = float(pos_dict['increment'])
                    coin_holdings = float(position['quantity_available'])


            # Get profile data
            profile_data = r.profiles.load_portfolio_profile()
            current_equity = profile_data['equity']
            current_equity = float(current_equity)
            previous_equity = profile_data['adjusted_equity_previous_close']
            previous_equity = float(previous_equity)
            daily_profit = float(current_equity) - float(previous_equity)

            # convert all the numerical values in historical_prices to float
            historical_prices = [float(x) for x in historical_prices]

            # Append data to dataframe
            crypto_data = crypto_data.append({
                'ticker': crypto,
                'current_price': current_price,
                'historical_prices': historical_prices,
                'buying_power': buying_power,
                'min_order_size': min_order_size,
                'coin_holdings': coin_holdings,
                'updated_at': now,
                'current_equity': current_equity,
                'previous_equity': previous_equity,
                'daily_profit': daily_profit
            }, ignore_index=True)
        except Exception as e:
            print(f'Error getting data for {crypto}: {e}')
            logger.error(f'Error getting data for {crypto}: {e}')
            traceback.print_exc()
            pass

    def add_me(crypto_data, metric, metric_name):
        if isinstance(metric, pd.Series):
            # if rsi is a series, get the last value
            crypto_data.loc[index, metric_name] = metric.iloc[-1]
        elif isinstance(metric, float):
            # if rsi is a float, just use it
            crypto_data.loc[index, metric_name] = metric
        elif isinstance(metric, np.ndarray):
            # if rsi is an array, get the last value
            crypto_data.loc[index, metric_name] = metric[-1]
        else:
            # otherwise, just use the value
            crypto_data.loc[index, metric_name] = metric
        return crypto_data

    # Calculate RSI, MACD, Bollinger Bands, etc. and add to dataframe
    for index, row in tqdm(crypto_data.iterrows()):
        prices = pd.Series(row['historical_prices'])
        # the ema below expects a series of floats, so convert if necessary
        if not isinstance(prices[0], float):
            prices = prices.apply(lambda x: float(x))
        # Calculate RSI (Relative Strength Index)
        # first make a copy of the data frame twice
        up_df, down_df = prices.copy(), prices.copy()
        # For up days, if the price is lower than previous price, set price to 0.
        up_df[up_df < up_df.shift(1)] = 0
        # For down days, if the price is higher than previous price, set price to 0.
        down_df[down_df > down_df.shift(1)] = 0
        # We need change and average gain for the first calculation.
        # Calculate the 1st-day change and average gain
        up_df.iloc[0] = 0
        down_df.iloc[0] = 0
        # Calculate the EWMA (Exponential Weighted Moving Average)
        roll_up1 = up_df.ewm(span=14).mean()
        roll_down1 = down_df.ewm(span=14).mean()
        # Calculate the RSI based on EWMA
        RS1 = roll_up1 / roll_down1
        RSI1 = 100.0 - (100.0 / (1.0 + RS1))
        rsi = RSI1 # save it for later
        
        # Calculate MACD (Moving Average Convergence Divergence)
        exp1 = prices.ewm(span=12, adjust=False).mean()
        # Calculate the 26-day EMA
        exp2 = prices.ewm(span=26, adjust=False).mean()
        # Subtract 26-day EMA from 12-day EMA, and you will get the first MACD
        macd = exp1 - exp2
        # Calculate the 9-day EMA of the MACD above
        exp3 = macd.ewm(span=9, adjust=False).mean()
        # Plot the MACD and exp3
        macd = macd - exp3
        # save it for later
        macd = macd.iloc[-1]
        macd_signal = exp3.iloc[-1] # the signal line
        macd_hist = macd - macd_signal # the histogram
        ma200 = prices.rolling(200).mean().iloc[-1] # 200 day moving average
        ma50 = prices.rolling(50).mean().iloc[-1] # 50 day moving average
        ma20 = prices.rolling(20).mean().iloc[-1] # 20 day moving average
        sma = prices.rolling(5).mean().iloc[-1] # 5 day simple moving average
        ema = prices.ewm(span=5, adjust=False).mean().iloc[-1] # 5 day exponential moving average

        # Calculate Bollinger Bands
        # Calculating the Upper and Lower Bands
        # We will be using a 14 day window
        window = 14
        # Calculate rolling mean and standard deviation using number of days set above
        rolling_mean = prices.rolling(window).mean()
        rolling_std = prices.rolling(window).std()
        # Create two new DataFrame columns to hold values of upper and lower Bollinger bands
        upper_band = pd.Series()
        upper_band = rolling_mean + (rolling_std * 2)
        lower_band = rolling_mean - (rolling_std * 2)

        # Add to dataframe
        print(f'Adding technical indicators for {row["ticker"]} to dataframe\n\t...')
        #^ Add the RSI
        crypto_data = add_me(crypto_data, rsi, 'rsi')
        #^ Add the MACD
        crypto_data = add_me(crypto_data, macd, 'macd')
        #^ Add the SMA from the Bollinger Bands
        crypto_data = add_me(crypto_data, rolling_mean, 'sma')
        #^ Add the EMA from the Bollinger Bands
        crypto_data = add_me(crypto_data, ema, 'ema')
        #^ Add the 200 day moving average
        crypto_data = add_me(crypto_data, ma200, 'ma200')
        #^ Add the 50 day moving average
        crypto_data = add_me(crypto_data, ma50, 'ma50') 
        #^ Add the 20 day moving average
        crypto_data = add_me(crypto_data, ma20, 'ma20')
        #^ Add the 5 day simple moving average
        crypto_data = add_me(crypto_data, sma, 'sma')
        #^ Add the 5 day exponential moving average
        crypto_data = add_me(crypto_data, ema, 'ema')
        #^ Add the MACD signal line
        crypto_data = add_me(crypto_data, macd_signal, 'macd_signal')
        #^ Add the MACD histogram
        crypto_data = add_me(crypto_data, macd_hist, 'macd_hist')
        #^ Add the Upper Band
        crypto_data = add_me(crypto_data, upper_band, 'upper_band')
        #^ Add the Lower Band
        crypto_data = add_me(crypto_data, lower_band, 'lower_band')
    # save the crypto_data dataframe to a csv file
    crypto_data.to_csv('crypto_data.csv', index=False)
    return crypto_data


ModuleNotFoundError: No module named 'robin_stocks'

In [ ]:
from colorama import Fore, Back, Style
color_options = [
    Fore.BLACK,
    Fore.RED,
    Fore.GREEN,
    Fore.YELLOW,
    Fore.BLUE,
    Fore.MAGENTA,
    Fore.CYAN,
    Fore.WHITE,
    Fore.LIGHTBLACK_EX,
    Fore.LIGHTRED_EX,
    Fore.LIGHTGREEN_EX,
    Fore.LIGHTYELLOW_EX,
    Fore.LIGHTBLUE_EX,
    Fore.LIGHTMAGENTA_EX,
    Fore.LIGHTCYAN_EX
]
background_options = [
    Back.BLACK,
    Back.RED,
    Back.GREEN,
    Back.YELLOW,
    Back.BLUE,
    Back.MAGENTA,
    Back.CYAN,
    Back.WHITE,
    Back.LIGHTBLACK_EX,
    Back.LIGHTRED_EX,
    Back.LIGHTGREEN_EX,
    Back.LIGHTYELLOW_EX,
    Back.LIGHTBLUE_EX,
    Back.LIGHTMAGENTA_EX,
    Back.LIGHTCYAN_EX
]
style_options = [
    Style.DIM,
    Style.NORMAL,
    Style.BRIGHT,
]

In [ ]:
def update_signals():
    stop_loss = 0.95 # 5% loss
    take_profit = 1.03 # 3% profit
    global crypto_data # make it global so we can update it
    crypto_data = pd.read_csv('crypto_data.csv')
    try:
        # Generate trading signals based on technical indicators
        for index, row in tqdm(crypto_data.iterrows()):
            # Convert historical_prices to a list if it's not already
            if isinstance(row['historical_prices'], str):
                historical_prices = [float(price) for price in row['historical_prices'].strip('[]').split(',')]
            else:
                historical_prices = [float(x) for x in row['historical_prices']]

            # convert all values to floats in the row that are numbers
            for col in crypto_data.columns[1:]:
                try:
                    # drop NaN values
                    if row[col] == 'NaN':
                        crypto_data.loc[index, col] = np.nan
                    else:
                        crypto_data.loc[index, col] = float(row[col])
                except:
                    pass

            historical_prices_last_5 = historical_prices[-5:]
            # Calculate the percent change over the last 5 hours
            if all(price != 0.0 for price in historical_prices_last_5):
                # if all the values are not zero, calculate the percent change
                pct_change_5h = (float(row['current_price']) - historical_prices_last_5[-1]) / historical_prices_last_5[-1]
            else:
                # calculate the percent change from the last non-zero value
                try:
                    for price in historical_prices_last_5:
                        if price != 0.0:
                            pct_change_5h = (float(row['current_price']) - price) / price
                            break
                except:
                    pct_change_5h = 0.0  # or handle the zero values differently

            # ─── Signal Calculations ──────────────────────────────


            #print(f'pct_change_5h: {pct_change_5h}')
            crypto_data.loc[index, 'signal'] = 0  # set the default value of signal to 0
            # Check the conditions for each signal
            if (float(row['rsi']) < 30) and (float(row['macd']) > 0):  # if the rsi dips below 30 indicating oversold and the macd is positive, buy
                # if the signal is a buy background is green and text should be black
                print(Fore.BLACK + f'BUY SIGNAL: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] +=1
            elif (float(row['rsi']) > 70) and (float(row['macd']) < 0):  # if the rsi goes above 70 indicating overbought and the macd is negative, sell
                print(Fore.GREEN + f'SELL SIGNAL: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] -= -1
            elif (float(row['macd']) > float(row['macd_signal'])) and (float(row['current_price']) > float(row['sma'])):  # if the macd line crosses above the signal line and the price is above the SMA, buy
                print(Fore.YELLOW + f'BUY SIGNAL: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] += 1
            elif (float(row['macd']) < float(row['macd_signal'])) and (float(row['current_price']) < float(row['sma'])):  # if the macd line crosses below the signal line and the price is below the SMA, sell
                print(Fore.RED + f'SELL SIGNAL: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] += -1
            elif (float(row['current_price']) > float(row['ma200'])):  # if the price crosses above the 200-day moving average, buy
                print(Fore.BLUE + f'BUY SIGNAL: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] += 1
            elif (float(row['current_price']) < float(row['ma200'])):  # if the price crosses below the 200-day moving average, sell
                print(Back.GREEN + f'SELL SIGNAL: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] += -1
            elif (pct_change_5h > 0.05):  # if the price has increased by more than 5% in the last 5 hours, buy
                print(Back.MAGENTA + f'BUY SIGNAL: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] += 1
            elif (pct_change_5h < -0.05):  # if the price has decreased by more than 5% in the last 5 hours, sell
                print(Fore.LIGHTMAGENTA_EX + f'SELL SIGNAL: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] += -1
            else:
                crypto_data.loc[index, 'signal'] = 0
            
            # consider limit loss and take profit here
            # if the value of the coin has risen by the take profit percentage, sell
            if (float(row['current_price']) / float(row['buy_price'])) > take_profit:
                print(Back.BLACK + Fore.GREEN + f'TAKE PROFIT: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] -= -1
            elif (float(row['current_price']) / float(row['buy_price'])) < stop_loss:
                print(Back.BLACK + Fore.RED + f'STOP LOSS: {row["ticker"]}' + Fore.RESET)
                crypto_data.loc[index, 'signal'] -=1
            
            # using stop_loss and take_profit variables
        # save the crypto_data dataframe to a csv file
        crypto_data.to_csv('crypto_data.csv', index=False)
        return crypto_data
    except Exception as e:
        crypto_data.loc[index, 'signal'] = 0
        print(f'Error: {e}')
        return crypto_data


In [ ]:

crypto_data = get_crypto_data(cryptos, crypto_data)

crypto_data.to_csv('crypto_data.csv', index=False)
# show the dataframe
crypto_data.head()

  0%|          | 0/18 [00:00<?, ?it/s]

found 18 positions.


  6%|▌         | 1/18 [00:01<00:32,  1.88s/it]

found 18 positions.


 11%|█         | 2/18 [00:03<00:26,  1.66s/it]

...

 61%|██████    | 11/18 [00:17<00:11,  1.66s/it]

found 18 positions.


 67%|██████▋   | 12/18 [00:19<00:09,  1.62s/it]

found 18 positions.


 72%|███████▏  | 13/18 [00:20<00:07,  1.51s/it]

In [ ]:
dir(r.orders)

NameError: name 'r' is not defined

In [ ]:
import time
import datetime
from ratelimit import limits, sleep_and_retry

# @sleep_and_retry
# def sell_coin(coin, current_price, buying_power, volume, crypto_positions):
#     # use robin stocks to sell the coin
#     # if we are selling everything then cancel all other sell orders first
#     if volume == crypto_positions[coin]:
#         r.cancel_all_crypto_orders(coin)
#     # sell the coin
#     r.order_sell_crypto_by_quantity(
#         symbol=coin,
#         quantity=volume,
#         timeInForce='gtc',
#         jsonify=True
#     )
#     print(f'Selling {volume} {coin} at {current_price}...')
#     # calculate the new buying power
#     buying_power += current_price * volume
#     # return the new buying power
#     time.sleep(1)
#     return buying_power

# @sleep_and_retry
# def buy_coin(coin, current_price, buying_power, dollars_spent):
#     # use robin stocks to buy the coin
#     r.order_buy_crypto_by_price(
#         symbol=coin,
#         amountInDollars=dollars_spent,
#         timeInForce='gtc',
#         jsonify=True
#     )
#     print(f'Buying {dollars_spent} {coin} at {current_price}...')
#     # calculate the new buying power
#     buying_power -= dollars_spent
#     # return the new buying power
#     time.sleep(1)
#     return buying_power


In [ ]:
def order_crypto(symbol, quantity_or_price, amount_in='dollars', side='buy'):
    """
    The order_crypto function is used to place a crypto order.

    :param symbol: Specify the coin you want to buy
    :param quantity_or_price: Specify the amount of crypto you want to buy or sell
    :param amount_in: Specify the amount of money you want to spend on buying a coin
    :param side: Specify whether you want to buy or sell the coin
    :return: A dictionary with the following keys:
    :doc-author: Trelent
    """

    if side == 'buy':
        amountIn = 'dollars'
    else:
        amountIn = 'quantity'

    # cast the quantity_or_price to a string
    quantity_or_price = str(quantity_or_price)
    
    try:
        # use robin stocks to buy the coin
        r.orders.order_crypto(
            symbol=str(symbol),
            quantityOrPrice=float(quantity_or_price),
            amountIn=str(amount_in), # either 'quantity' or 'dollars'
            side=str(side),
            timeInForce='gtc',
            jsonify=True
        )
    except Exception as e:
        raise e

    

In [ ]:


buying_limiter = 0.60 # only use 60% of the buying power

# Read the crypto_data.csv file with the correct data types
crypto_data = pd.read_csv('crypto_data.csv', converters={'historical_prices': eval})
buying_power = float(r.profiles.load_account_profile(info='crypto_buying_power')) * float(buying_limiter) # only use 60% of the buying power

def is_daytime():
    # CHECK TIME OF DAY IN CENTRAL STANDARD TIME
    current_time = datetime.datetime.now()
    if current_time.hour >= 11 and current_time.hour <= 23:
        return True
    else:
        return False

print(f'I can only play with {buying_power} dollars')

# def main_loop():
#     """
#     The main_loop function is the main function of this program. It runs every ten minutes and updates the signals for each coin, then checks if we own any coins or not. If we do, it will check to see if there are any sell signals (signal &lt; -0.5) and sell 70% of our holdings in that coin; otherwise, it will check to see if there are buy signals (signal &gt; 0.5) and buy 50% of 1% of our buying power worth in that coin.
    
#     :return: The following:
#     :doc-author: Trelent
#     """
#     global crypto_data
#     global buying_limiter
#     # cast the numeric columns to float in the crypto_data dataframe
#     for column in ['signal', 'current_price', 'buy_price', 'buy_volume', 'sell_volume']:
#         try:
#             crypto_data[column] = crypto_data[column].astype(float)
#         except:
#             pass
#     buying_power = float(r.profiles.load_account_profile(info='crypto_buying_power')) * buying_limiter # only use 60% of the buying power
    
#     while True:
#         # cancel all orders
#         r.cancel_all_crypto_orders()
#         time.sleep(10)
        
#         # CHECK TIME OF DAY IN CENTRAL STANDARD TIME
#         current_time = datetime.datetime.now()
    
#         # IF ITS AFTER 11PM AND BEFORE 5AM, THEN THE MARKET IS CLOSED SO ONLY DEAL WITH CRYPTO, ALSO ALL SIGNALS SHOULD HAVE A REDUCED BUY_VOLUME BY REDUCING BUY_IN BY 90% KEEPING SELL VOLUMES THE SAME.
#         if current_time.hour >= 23 or current_time.hour < 5:
#             buying_limiter = 1 # 100% of buying power during the day
#             pass
#         else:
#             buying_limiter = 0.02 # 2% of buying power at night
        
        
#         crypto_data = get_crypto_data(cryptos, crypto_data)
#         try:
#             # Update signals
#             # Replace with your own function or code to update signals
#             update_signals()

#             # Get the current time
#             now = datetime.datetime.now()

#             for index, row in crypto_data.iterrows():
#                 coin = row['ticker']
#                 signal = row['signal']
#                 current_price = float(row['current_price'])
#                 buying_power = float(row['buying_power'])
#                 coin_holdings = float(row['coin_holdings'])
#                 try:
#                     crypto_positions = row['crypto_positions']
#                 except:
#                     crypto_positions = coin_holdings
#                 print(f' >> {row["ticker"]} << signal: {row["signal"]}, holding: {row["coin_holdings"]}, ${buying_power}')

#                 try:
#                     # Check if we own the coin
#                     if coin_holdings > 0:
#                         pass
#                     else:
#                         # We do not own the coin
#                         # buy 0.25 USD worth of the coin if the signal is 1
#                         try:
#                             if signal == 1:
#                                 # use order_crypto to buy the coin
#                                 order_crypto(
#                                     symbol = coin,
#                                     quantity_or_price = 0.25 * buying_limiter,
#                                     amount_in = 'dollars',
#                                     side = 'buy'
#                                 )
#                             elif signal > 1:
#                                 # Buy 1% for every number over 1
#                                 # use order_crypto to buy the coin
#                                 order_crypto(
#                                     symbol = coin,
#                                     quantity_or_price = (0.01 * (signal - 1) + 0.01) * buying_power * buying_limiter,
#                                     amount_in = 'dollars',
#                                     side = 'buy'
#                                 )
#                         except Exception as e:
#                             print(f'Error: {e}')
#                             logger.error(f"An error occurred: {e}, {traceback.format_exc()}, {dir(e)}")
#                             continue

#                     # We own the coin
#                     # Case One: The price is below the lowest value in the last five hours, meaning the price is dropping and we should sell the coin if we bought it at a higher price, or buy the coin if we don't own it
#                     list_prices = row['historical_prices']
#                     list_five_latest = list_prices[-5:]
#                     # cast the list to floats
#                     list_five_latest = [float(i) for i in list_five_latest]
#                     minimum = min(list_five_latest)
#                     if current_price < minimum:
#                         # Price is below the lowest value in the last five hours
#                         if coin_holdings > 0:
#                             # We own the coin
#                             # Sell 100% of the holdings
#                             # use order_crypto to sell the coin
#                             order_crypto(
#                                 symbol = coin,
#                                 quantity_or_price = coin_holdings,
#                                 amount_in = 'quantity',
#                                 side = 'sell'
#                             )
#                             #log it 
#                             logger.info(f' [!] ---> Selling {coin_holdings} {coin} at {current_price}...')
#                             # calculate the new buying power
#                             buying_power += current_price * coin_holdings
#                         else:
#                             # We do not own the coin
#                             # Calculate the amount of dollars to spend
#                             dollars_spent = 0.005 * buying_power
#                             print(f'Buying {dollars_spent} {coin} at {current_price}...')
#                             # Buy the coin
#                             # use order_crypto to buy the coin
#                             order_crypto(
#                                 symbol=coin,
#                                 quantity_or_price=dollars_spent,
#                                 amount_in='dollars',
#                                 side='buy'
#                             )
#                             #log it 
#                             logger.info(Fore.Green + f' [!] ---> Buying {dollars_spent} {coin} at {current_price}...')
#                             buying_power -= dollars_spent
#                     if signal == -1:
#                         # Sell 70% of the holdings
#                         # use order_crypto to sell the coin
#                         order_crypto(
#                             symbol=coin,
#                             quantity_or_price=0.7 * coin_holdings,
#                             amount_in='quantity',
#                             side='sell'
#                         )
#                         # calculate the new buying power
#                         buying_power += current_price * coin_holdings
#                     if signal < -1:
#                         # Sell 100% of the holdings
#                         # use order_crypto to sell the coin
#                         order_crypto(
#                             symbol = coin,
#                             quantity_or_price = coin_holdings,
#                             amount_in = 'quantity',
#                             side = 'sell'
#                         )
#                         #log it 
#                         logger.info(f' [!] ---> Selling {coin_holdings} {coin} at {current_price}...')
#                         # calculate the new buying power
#                         buying_power += current_price * coin_holdings
#                     if signal == 1:
#                         # Buy 50% of 1% of the buying power
#                         # use r.orders to buy the coin
#                         order_crypto(
#                             symbol = coin,
#                             quantity_or_price = 0.005 * buying_power,
#                             amount_in = 'dollars',
#                             side = 'buy'
#                         )
#                         #log it 
#                         logger.info(f'Buying {0.005 * buying_power} {coin} at {current_price}...')
                    
#                         print(f'Buying {0.005 * buying_power} {coin} at {current_price}...')
#                         # reduce the buying power
#                         buying_power -= 0.005 * buying_power
#                     if signal > 1:
#                         # Buy 1% for every number over 1
#                         # use r.orders to buy the coin
#                         order_crypto(
#                             symbol = coin,
#                             quantity_or_price = (0.01 * (signal - 1) + 0.01) * buying_power,
#                             amount_in = 'dollars',
#                             side = 'buy'
#                         )
#                         #log it
#                         logger.info(f'Buying {(0.01 * (signal - 1) + 0.01) * buying_power} {coin} at {current_price}...')
#                         print(f'Buying {(0.01 * (signal - 1) + 0.01) * buying_power} {coin} at {current_price}...')
#                         ic()
#                         # reduce the buying power
#                         buying_power -= (0.01 * (signal - 1) + 0.01) * buying_power
#                 except Exception as e:
#                     logger.error(f"An error occurred: {e}, {traceback.format_exc()}, {dir(e)}")
#                     print(f'Error: {e}')
#                     continue


#         except Exception as e:
#             logger.error(f"An error occurred: {e}, {traceback.format_exc()}, {dir(e)}")
#                 # Wait for ten minutes and show a loading bar
#         # print out a detailed synopsis of the results so far
#         print(f'Current holdings: {r.get_crypto_positions()}')
#         print(f'Current buying power: {r.profiles.load_account_profile(info="crypto_buying_power")}')
#         # profits
#         print(f'Current profits: {r.profiles.load_account_profile(info="crypto_day_traded_profit_loss")}')
#         # wait for ten minutes
#         # if daytime then wait 5 minutes
#         # if night time then wait 30 minutes
#         if is_daytime():
#             print('Waiting 5 minutes...')
#             for i in tqdm(range(60*5)):
#                 time.sleep(1)
#         else:
            
#             print('Waiting 30 minutes...')
#             for i in tqdm(range(60*30)):
#                 time.sleep(1)


I can only play with 15.222 dollars


In [ ]:
def main_loop():
    """
    The main_loop function is the main function of this program. It runs every ten minutes and updates the signals for each coin, then checks if we own any coins or not. If we do, it will check to see if there are any sell signals (signal &lt; -0.5) and sell 70% of our holdings in that coin; otherwise, it will check to see if there are buy signals (signal &gt; 0.5) and buy 50% of 1% of our buying power worth in that coin.
    
    :return: The following:
    :doc-author: Trelent
    """
    global crypto_data
    
    while True:
        # cancel all orders
        r.cancel_all_crypto_orders()
        time.sleep(10)
        
        # CHECK TIME OF DAY IN CENTRAL STANDARD TIME
        current_time = datetime.datetime.now()
    
        # IF ITS AFTER 11PM AND BEFORE 5AM, THEN THE MARKET IS CLOSED SO ONLY DEAL WITH CRYPTO, ALSO ALL SIGNALS SHOULD HAVE A REDUCED BUY_VOLUME BY REDUCING BUY_IN BY 90% KEEPING SELL VOLUMES THE SAME.
        buying_limiter = 1 if current_time.hour >= 23 or current_time.hour < 5 else 0.02
        
        crypto_data = get_crypto_data(cryptos, crypto_data)
        
        try:
            update_signals()
            now = datetime.datetime.now()

            for index, row in tqdm(crypto_data.iterrows()):
                coin = row['ticker']
                signal = row['signal']
                current_price = float(row['current_price'])
                buying_power = float(row['buying_power'])
                coin_holdings = float(row['coin_holdings'])
                crypto_positions = coin_holdings

                print(f' >> {row["ticker"]} << signal: {row["signal"]}, holding: {row["coin_holdings"]}, ${buying_power}')

                if coin_holdings > 0 and signal < 0:
                    # We own the coin and signal indicates selling
                    quantity = 0.7 * coin_holdings if signal == -1 else coin_holdings
                    order_crypto(symbol=coin, quantity_or_price=quantity, amount_in='quantity', side='sell')
                    buying_power += current_price * quantity
                    logger.info(Fore.RED + f' [!] ---> Selling {quantity} {coin} at {current_price}...' + Fore.RESET)
                elif signal > 0:
                    # Signal indicates buying
                    amount = 0.005 * buying_power if signal == 1 else (0.01 * (signal - 1) + 0.01) * buying_power
                    order_crypto(symbol=coin, quantity_or_price=amount, amount_in='dollars', side='buy')
                    buying_power -= amount
                    logger.info(Fore.GREEN + f' [!] ---> Buying {amount} {coin} at {current_price}...' + Fore.RESET)

                # Handling lowest value in last 5 hours condition
                list_prices = row['historical_prices']
                list_five_latest = list_prices[-5:] #todo what units are these in?
                minimum = min(list_five_latest)
                if isinstance(minimum, str):
                    minimum = float(minimum)
                    logging.info(f'Converting minimum: {minimum} to float... it was a string')
                if isinstance(minimum, list):
                    minimum = float(minimum[0])
                    logging.info(f'Converting minimum: {minimum} to float... it was a list')
                else:
                    minimum = float(minimum)
                    logging.info(f'Converting minimum: {minimum} to float... it was a string')
                if current_price < minimum:
                    if coin_holdings > 0:
                        order_crypto(symbol=coin, quantity_or_price=coin_holdings, amount_in='quantity', side='sell')
                        buying_power += current_price * coin_holdings
                        logger.info(Fore.RED + f' [!] ---> Selling {coin_holdings} {coin} at {current_price}...' + Fore.RESET)
                    else:
                        dollars_spent = 0.005 * buying_power
                        order_crypto(symbol=coin, quantity_or_price=dollars_spent, amount_in='dollars', side='buy')
                        buying_power -= dollars_spent
                        logger.info(Fore.GREEN + f' [!] ---> Buying {dollars_spent} {coin} at {current_price}...' + Fore.RESET)
            tqdm.write(f'we have {buying_power} left')
        except Exception as e:
            logger.error(f"An error occurred: {e}, {traceback.format_exc()}, {dir(e)}")

        # Wait for ten minutes and show a loading bar
        print(f'Current holdings: {r.get_crypto_positions()}')
        print(f'Current buying power: {r.profiles.load_account_profile(info="crypto_buying_power")}')
        print(f'Current profits: {r.profiles.load_account_profile(info="crypto_day_traded_profit_loss")}')
        
        # if daytime then wait 5 minutes
        # if night time then wait 30 minutes
        wait_time = 5 if is_daytime() else 30
        print(f'Waiting {wait_time} minutes...')
        for i in tqdm(range(60*wait_time)):
            time.sleep(1)


In [ ]:
# emergency sell all crypto
if panic_mode:
    for index, row in crypto_data.iterrows():
        coin = row['ticker']
        coin_holdings = float(row['coin_holdings'])
        if coin_holdings > 0:
            r.order_sell_crypto_by_quantity(
                symbol=coin,
                quantity=coin_holdings,
                timeInForce='gtc',
                jsonify=True
            )
            print(f'Selling {coin_holdings} {coin}...')
        else:
            print(f'No holdings for {coin}...')
        

In [ ]:
main_loop()

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Loading page 28 ...
Loading page 29 ...
Loading page 30 ...
Loading page 31 ...
Loading page 32 ...
Loading page 33 ...
Loading page 34 ...
Loading page 35 ...
Loading page 36 ...
Loading page 37 ...
Loading page 38 ...
Loading page 39 ...
Loading page 40 ...
Loading page 41 ...
Loading page 42 ...
Loading page 43 ...
Loading page 44 ...
Loading page 45 ...
Loading page 46 ...
Loading page 47 ...
Loading page 48 ...
Loading page 49 ...
Loading page 50 ...
Load

  0%|          | 0/18 [00:00<?, ?it/s]

found 18 positions.


  6%|▌         | 1/18 [00:01<00:26,  1.57s/it]

found 18 positions.


...

0it [00:00, ?it/s]

 >> BTC << signal: 0.0, holding: 0.000545, $25.37
 >> ETH << signal: nan, holding: 0.002215, $25.37
 >> ADA << signal: nan, holding: 0.11, $25.37
 >> DOGE << signal: nan, holding: 211.16, $25.37
 >> MATIC << signal: nan, holding: 0.0, $25.37


INFO:crypto_trader: [!] ---> Buying 0.12685000000000002 MATIC at 0.723236843905465...
5it [00:01,  3.83it/s]
ERROR:crypto_trader:An error occurred: could not convert string to float: '-', Traceback (most recent call last):
  File "/var/folders/34/d1tlq3k91hb0lj6x90xpzb4r0000gn/T/ipykernel_97630/453296569.py", line 55, in main_loop
    minimum = float(minimum.replace(',', ''))
ValueError: could not convert string to float: '-'
, ['__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', '__traceback__', 'args', 'with_traceback']


 >> SHIB << signal: nan, holding: 100747.0, $25.37
Current holdings: [{'account_id': '61f33e4c-8cc5-497e-a296-f0f9691a030e', 'cost_bases': [{'currency_id': '1072fc76-1862-41ab-82c2-485837590762', 'direct_cost_basis': '0.037672606713634480', 'direct_quantity': '0.110000000000000000', 'direct_reward_cost_basis': '0.000000000000000000', 'direct_reward_quantity': '0.000000000000000000', 'direct_transfer_cost_basis': '0.000000000000000000', 'direct_transfer_quantity': '0.000000000000000000', 'id': '6358937c-6d30-46e3-b76b-abc2bf2c7834', 'intraday_cost_basis': '0.000000000000000000', 'intraday_quantity': '0.000000000000000000', 'marked_cost_basis': '0.000000000000000000', 'marked_quantity': '0.000000000000000000'}], 'created_at': '2022-10-25T21:55:08.627884-04:00', 'currency': {'brand_color': '', 'code': 'ADA', 'crypto_type': 'BASE_ASSET', 'display_only': False, 'id': 'f5cd90c4-6058-4e2c-909d-2fe46bb80caf', 'increment': '0.000001000000000000', 'name': 'Cardano', 'type': 'cryptocurrency'}, 'i

  3%|▎         | 10/300 [00:10<04:51,  1.00s/it]